# Principles of Digital Communications - Project

The project is divided in 3 milestones:
- milestone 1 is transforming a string into an array of bits, and an array of bits into a string (with utf-8 encoding)
- milestone 2 is creating a transmitter and receiver in the case of a noiseless channel
- milestone 3 is taking into account the white gaussian noise in our channel

In [1]:
import numpy as np

In [2]:
#we can run terminal commands in here if we start by '!'
! ls

PDC_project_5.ipynb    project_pdc_pres.ipynb
README.md              urn-model.ipynb


## All the utility functions: channels, bits <--> string, create codebook, bits <--> int

Simulates the channel with noise (milestone 3)

In [5]:
def channel(channel_input):
    channel_input = np.clip(channel_input,-1,1)
    erased_index = np.random.randint(3) 
    channel_input[erased_index:len(channel_input):3] = 0
    return channel_input + np.sqrt(10)*np.random.randn(len(channel_input))

Simulates the channel without noise (milestone 2)

In [6]:
def noiseless_channel(channel_input):
    channel_input = np.clip(channel_input,-1,1)
    erased_index = np.random.randint(3) 
    channel_input[erased_index:len(channel_input):3] = 0
    return channel_input

Converts a string to an array of bits (UTF-8 encoding)

In [7]:
def string_to_bits(string):
    arr = bytearray(string, 'utf-8')
    return np.unpackbits(arr)

Converts an array of bits to a string

In [8]:
def bits_to_string(bits):
    b = bytearray(np.packbits(bits))
    return str(b, 'utf-8', errors='replace')

Creates a codebook of size 2**k

In [9]:
def create_codebook(k):
    C_i0 = [[1]]
    #it is an iterative process
    if k == 0:
        return C_i0
    for i in range(0,k):
        C_i1 = []
        for c in C_i0:
            C_i1.append(np.concatenate((c,c)))
            C_i1.append(np.concatenate((c,np.dot(-1,c))))
        C_i0 = C_i1
    return np.array(C_i1)

Converts some bits to an int (unsigned)

In [10]:
def bin_to_int(bits):
    a = np.flip(bits)
    result = 0
    for k in range(0,len(a)):
        result += a[k]*(2**k)
    return result

Convert an integer to an array of bits

In [11]:
def int_to_bits(number, size):
    result = np.zeros(size, dtype = int)
    b = bin(number)
    i = len(b)-1
    j = 0
    while i > 1:
        result[j] = int(b[i])
        i-=1
        j+=1
    result = result[:size]
    return np.flip(result)

# Mod3 Scheme and Orthogonal Codes

First: bits to the mod3 scheme: first half, second half, parity_check (xor)

In [12]:
#bits is of length 8*80 = 640
def mod3_scheme(bits):
    bits = np.array(bits)
    if len(bits) < 8*80:
        bits.resize(8*80)
    k = int(len(bits)/2)
    v1 = bits[:k]
    v2 = bits[k:]
    parity_check = np.bitwise_xor(v1,v2)
    return [v1,v2,parity_check]
#shape: (3,320)

Then we translate the 3 vectors to their corresponding codeword collection. In this step, there is some tuning to do

In [13]:
#helper method to translate an array to the corresponding codewords
def array_to_codewords(array, word_size, codebook):
    result = []
    n_words = int(len(array)/word_size)
    for i in range(0,n_words):
        u = int(word_size*i)
        v = int(word_size*i+word_size)
        b = array[u:v]
        index = bin_to_int(b)
        result.append(codebook[index])
    return np.array(result).flatten()
#the result should be of legth n_words*2**k and must be <= 20000

In [14]:
#n_words    word_size      k         total_n
# 64      |     5     |    8     |   49152         --> best error prob about 0.4
# 40      |     8     |    8     |   30720         --> bad error prob about 0.75

ws = 5
nw = int(320/ws)
K = 8
n = (2**K)*3*nw
    
#array is of length 960
def orthogonal_encoder(array, word_size=ws, k=K):
    m = len(array[0]) #should be 320
    codebook = create_codebook(k)
    v1 = array_to_codewords(array[0], word_size, codebook)
    v2 = array_to_codewords(array[1], word_size, codebook)
    v3 = array_to_codewords(array[2], word_size, codebook)
    result = []
    #here, we append them to an array alternatively
    for i in range(0,len(v1)):
        result.append(v1[i])
        result.append(v2[i])
        result.append(v3[i])
    return np.array(result)
#the result should be of length 3*n_words*(2**k) and must be <= 60000
n

49152

Guess H by taking argmin of the squared-norm of 3 vectors

In [15]:
def H_guesser(array):
    #argmin_j |y_j|^2
    a1 = array[0::3]
    a2 = array[1::3]
    a3 = array[2::3]
    y1 = (np.linalg.norm(a1))
    y2 = (np.linalg.norm(a2))
    y3 = (np.linalg.norm(a3))
    return np.argmin([y1,y2,y3])

Now that we know H, we try to recreate the two vectors that went through

In [16]:
def decode_word(word, codebook, word_size):
    #if word_size <= k, no need to check the whole codebook
    p = 2**word_size
    r = [np.linalg.norm(c-word) for c in codebook[:p]]
    return np.argmin(r)
        
def decode_array(array, n_words, word_size, codebook, k):
    m = 2**k
    result = []
    for i in range(0,n_words):
        word = array[m*i:m*i+m]
        result.append(int_to_bits(decode_word(word,codebook,word_size), word_size))
    return np.array(result).flatten()

In [17]:
def orthogonal_decoder(array, H, word_size=ws, k=K, n_words=nw):
    codebook = create_codebook(k)
    v1 = array[0::3]
    v2 = array[1::3]
    v3 = array[2::3]
    
    if H==0:
        #decode v2 and v3
        b1 = np.zeros(n_words*word_size, dtype=int)
        b2 = decode_array(v2, n_words, word_size, codebook, k)
        b3 = decode_array(v3, n_words, word_size, codebook, k)
    elif H==1:
        #decode v1 and v3
        b2 = np.zeros(n_words*word_size, dtype=int)
        b1 = decode_array(v1, n_words, word_size, codebook, k)
        b3 = decode_array(v3, n_words, word_size, codebook, k)
    else:
        #decode v1 and v2
        b3 = np.zeros(n_words*word_size, dtype=int)
        b1 = decode_array(v1, n_words, word_size, codebook, k)
        b2 = decode_array(v2, n_words, word_size, codebook, k)
    return [b1,b2,b3]


#result should be back to len = 960

In [18]:
#array is of length 960
def mod3_decoder(array, H):
    if H == 2:
        a1 = array[0]
        a2 = array[1]
    elif H==1:
        a1 = array[0]
        a3 = array[2]
        a2 = np.bitwise_xor(a1, a3)
    else:
        a2 = array[1]
        a3 = array[2]
        a1 = np.bitwise_xor(a2, a3)
    return np.concatenate((a1, a2))
#the result is of length 640

## Explanation of the communication scheme:
### 1. Convert the string to an array of bits:


### 2. Cut this array in half, create 3 arrays: 1st half, 2nd half, parity check (xor of the 2 first). That way, by only knowing 2 of these 3 arrays, we can reproduce the last one.


### 3. Assign each array to a list of codewords, obtained from orthogonal codes. Then put them together alternatively, so that when it goes through a channel deleting 1/3 inputs, we can retrieve 2 of these codeword lists.

For k = 2, this is what the codebook would look like:

$ \mathcal{C} = \{ \begin{bmatrix}1\\1\\1\\1\end{bmatrix}, \begin{bmatrix}1\\1\\-1\\-1\end{bmatrix}, \begin{bmatrix}1\\-1\\1\\-1\end{bmatrix}, \begin{bmatrix}1\\-1\\-1\\1\end{bmatrix} \}$


At this step, we can tune parameters to obtain the best result:
- $2^k$: the codeword size (and codebook length)
- ws and n_words: the word size and number of words, which define the minimum codebook size and the amount of codewords that will be transmitted

We obtain our block length n: $n = 2^k * 3 * $nw , which has to be less than 60'000.

### 4. The array of length n goes through the communication channel.

$ X = \begin{bmatrix} 1 & 1 & 1 & -1 & ... & -1 & -1 & 1\end{bmatrix}$ $\rightarrow$ _channel_ $\rightarrow$ $ Y = \begin{bmatrix} 3.43 & 2.24 & -7.32 & -3.2 & ... & -0.54 & 1.32 & 6.32\end{bmatrix}$


### 5. Guess H using a MAP estimator.

$ H = argmin_{i=1,2,3} ||Y_i||$ 

with $Y_i$'s being the 3 subarrays of $Y$, obtained by taking alternatively 1/3 elements.


### 6. Knowing H, retrieve the 2 lists of codewords (affected by the noise) that weren't deleted. Use a MAP estimator to estimate each word. Translate them back to an array of bits.

$ \hat{x_l} = argmin_{j=1,...,2^k} ||c_j - y_l||$ for $y_l$ words of $Y_i$'s and $c_j \in \mathcal{C}$

### 7. Now that we have 2 of the arrays, we can xor them together to obtain the missing one.

### 8. Finally, convert the array back to string, with utf-8 encoding. Then compare the string with the original one.

# Communication:


In [19]:
string80 = "I wish my sentence had length 80, otherwise my communication scheme would not..."

In [20]:
#if the string is given in txt:
#string80 = open('filename.txt', 'r').read()

In [21]:
# The local communication system
def comm(string=string80):
    a = string_to_bits(string)
    b = mod3_scheme(a)
    X = orthogonal_encoder(b)
    Y = channel(X)
    H = H_guesser(Y)
    c = orthogonal_decoder(Y,H)
    d = mod3_decoder(c, H)
    return bits_to_string(d)

In [22]:
comm()

'I wish my sentence had length 80, otherwise my communication scheme would not...'

In [27]:
# Method to approximate an error of probability
def error_prob():
    yes = 0
    total = 1000
    for i in range(0,total):
        if comm() == string80:
            yes+=1
    return 1 - (yes/total)

In [28]:
error_prob()

0.389

In [23]:
def mean_and_max():
    mean = 0
    maxi = 0
    maxi2 = 0
    total = 100
    for i in range(0,total):
        b = comm()
        s = sum(string80[i] != b[i] for i in range(len(string80)))
        mean += s
        if s>maxi:
            maxi2 = maxi
            maxi = s
    return mean/total, maxi, maxi2

In [24]:
mean_and_max()

(0.82, 11, 4)